<a href="https://colab.research.google.com/github/jeffersonramelo/Paper-pos-doc/blob/main/C%C3%B3pia_de_modelos_tradicionais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [144]:
import pandas as pd

# Caminho para o arquivo XLSX
caminho_arquivo = '/content/dados todos sem escala.xlsx'

# Lê o arquivo XLSX
dados = pd.read_excel(caminho_arquivo)

In [145]:
# Supondo que 'dados' seja o seu DataFrame

# Converter a coluna 't' para o tipo de dado datetime, se ainda não estiver no formato adequado
dados['t'] = pd.to_datetime(dados['t'], format='%Y')

# Definir as datas de início e fim para o intervalo desejado
data_inicio = pd.to_datetime('2017-01-01')
data_fim = pd.to_datetime('2022-12-31')

# Filtrar os dados para extrair apenas as linhas no intervalo de 2010 a 2013
dados = dados.query('@data_inicio <= t <= @data_fim')

# Agora, 'dados_2010_2013' contém apenas as linhas com valores de 't' no intervalo de 2010 a 2013.


In [117]:
dados

,COD,t,VM30Abril2011,PL2010,LL2010,AT2009
1382,EALT4,2017-01-01,1.237500e+05,97996.0,30648.0,244566.0
1383,AESB3,2017-01-01,4.512474e+06,1557690.0,298277.0,4099579.0
1384,AFLT3,2017-01-01,4.983715e+05,44935.0,12517.0,87616.0
1385,AALR3,2017-01-01,1.714960e+06,1219460.0,6652.0,2166312.0
1386,ALPA4,2017-01-01,7.564099e+06,2186390.0,362289.0,3782052.0
...,...,...,...,...,...,...
2849,WHRL4,2022-01-01,6.630416e+06,2176471.0,416715.0,9164610.0
2850,PORT3,2022-01-01,4.390289e+06,2345826.0,327064.0,5724443.0
2851,WLMM4,2022-01-01,7.129722e+05,622011.0,117298.0,668104.0
2852,YDUQ3,2022-01-01,2.358681e+06,2946247.0,-58244.0,9897795.0


excluir setor financeiro e empresas com PL negativo, numeros faltante e transformar eventuais caracteres em números

In [146]:
# Agora, vamos remover as linhas onde 'd9' seja igual a 1
#dados = dados[dados['d11'] != 1]
# Converte a coluna 'PL2010' para números (float) e remove as linhas onde 'PL2010' seja negativo
dados['PL2010'] = pd.to_numeric(dados['PL2010'], errors='coerce')  # 'coerce' trata strings não numéricas como NaN
dados = dados[dados['PL2010'] > 0]

# Agora, vamos remover as linhas onde 'VM30Abril11' seja igual a '-'
dados = dados[dados['VM30Abril2011'] != '-']
dados = dados[dados['PL2010'] != '-']
dados = dados[dados['LL2010'] != '-']
dados = dados[dados['AT2009'] != '-']
dados = dados[dados['AT2009'] != '0']

#transforma as variáveis em números
dados['AT2009'] = pd.to_numeric(dados['AT2009'], errors='coerce')
dados['LL2010'] = pd.to_numeric(dados['LL2010'], errors='coerce')
dados['VM30Abril2011'] = pd.to_numeric(dados['VM30Abril2011'], errors='coerce')
dados['PL2010'] = pd.to_numeric(dados['PL2010'], errors='coerce')

# Lista de variáveis que você deseja escalonar
variaveis = ['VM30Abril2011', 'PL2010', 'LL2010', 'AT2009']

# Verifique o tipo de dados de cada coluna
for coluna in variaveis:
    print(f"Tipo de dados da coluna {coluna}: {dados[coluna].dtype}")


Tipo de dados da coluna VM30Abril2011: float64
Tipo de dados da coluna PL2010: float64
Tipo de dados da coluna LL2010: float64
Tipo de dados da coluna AT2009: float64


ESCALONAMENTO DAS VARIÁVEIS

In [157]:
variaveis = ['VM30Abril2011', 'PL2010', 'LL2010']

for coluna in variaveis:
    dados[coluna] = dados[coluna] / dados['AT2009']


WINSORIZAÇÃO: tratar outliers: A winsorização ao nível de 1% envolve substituir os 1% dos valores mais extremos por valores no limite do intervalo aceitável, ou seja, os 1% menores e os 1% maiores dos valores.

In [147]:
import pandas as pd
import numpy as np
from scipy.stats import mstats

# Supondo que seu DataFrame se chame "dados"

# Lista de variáveis para winsorizar
variaveis_winsorizar = ['VM30Abril2011', 'PL2010', 'LL2010', 'AT2009']

# Converter as colunas para números, tratando não numéricos como NaN
for coluna in variaveis_winsorizar:
    dados[coluna] = pd.to_numeric(dados[coluna], errors='coerce')

# Aplicar winsorização de 1% nas variáveis
for coluna in variaveis_winsorizar:
    dados[coluna] = mstats.winsorize(dados[coluna], limits=[0.01, 0.01])

# Agora, as variáveis especificadas foram winsorizadas


MQO_SIMPLES

In [158]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_white

# Suponha que 'dados' seja o DataFrame com suas variáveis

# Defina as variáveis independentes e dependente
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Certifique-se de que os tipos de dados sejam numéricos
X = X.astype(float)
y = y.astype(float)

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Crie o modelo de regressão linear
modelo = sm.OLS(y, X).fit()

# Visualize os resultados do modelo
print(modelo.summary())

# Calcule o MedAE
y_pred = modelo.predict(X)
medae = np.median(np.abs(y - y_pred))

# Calcule o valor médio da variável de resposta
valor_medio_y = np.mean(y)

# Calcule o MedAE escalado pelo valor (ou preço)
medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem

print("Median Absolute Error (MedAE):", medae)
print("MedAE escalado pelo valor (ou preço) em percentagem:", medae_scaled)

# Calcular os VIFs (Fator de Inflação da Variância)
def calculate_vif(dataframe):
    vif_data = pd.DataFrame()
    vif_data["Variável"] = dataframe.columns
    vif_data["VIF"] = [variance_inflation_factor(dataframe.values, i) for i in range(dataframe.shape[1])]
    return vif_data

vif_result = calculate_vif(X.drop(columns=['const']))
print("\nFator de Inflação da Variância (VIF):")
print(vif_result)

# Realizar o teste de heterocedasticidade de White
white_test_result = het_white(modelo.resid, X)
print("\nResultado do teste de heterocedasticidade de White:")
print("LM Statistic:", white_test_result[0])
print("LM p-value:", white_test_result[1])
print("F Statistic:", white_test_result[2])
print("F p-value:", white_test_result[3])


                            OLS Regression Results                            
Dep. Variable:          VM30Abril2011   R-squared:                       0.525
Model:                            OLS   Adj. R-squared:                  0.524
Method:                 Least Squares   F-statistic:                     697.6
Date:                Fri, 29 Sep 2023   Prob (F-statistic):          8.01e-205
Time:                        23:52:32   Log-Likelihood:                -3132.2
No. Observations:                1266   AIC:                             6270.
Df Residuals:                    1263   BIC:                             6286.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.3902      0.113    -12.304      0.0

MQP - MODELO SIMPLES

In [149]:
import statsmodels.api as sm
import numpy as np
import pandas as pd

# Suponha que 'dados' seja o DataFrame com suas variáveis

# Defina as variáveis independentes e dependente
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Certifique-se de que os tipos de dados sejam numéricos
X = X.astype(float)
y = y.astype(float)

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Crie o modelo de regressão linear robusta usando o método WLS com pesos robustos
modelo_robusto = sm.RLM(y, X, M=sm.robust.norms.HuberT()).fit()

# Visualize os resultados do modelo robusto
print(modelo_robusto.summary())

# Calcule o MedAE
y_pred_robusto = modelo_robusto.predict(X)
medae_robusto = np.median(np.abs(y - y_pred_robusto))

# Calcule o valor médio da variável de resposta
valor_medio_y = np.mean(y)

# Calcule o MedAE escalado pelo valor (ou preço)
medae_scaled_robusto = (medae_robusto / valor_medio_y) * 100  # Em percentagem

print("Median Absolute Error (MedAE) robusto:", medae_robusto)
print("MedAE robusto escalado pelo valor (ou preço) em percentagem:", medae_scaled_robusto)

# Calcular os VIFs (Fator de Inflação da Variância)
def calculate_vif(dataframe):
    vif_data = pd.DataFrame()
    vif_data["Variável"] = dataframe.columns
    vif_data["VIF"] = [variance_inflation_factor(dataframe.values, i) for i in range(dataframe.shape[1])]
    return vif_data

vif_result = calculate_vif(X)
print("\nFator de Inflação da Variância (VIF):")
print(vif_result)

# Calcular o coeficiente de determinação (R-squared)
sst_robusto = ((y - y.mean())**2).sum()
ssr_robusto = ((y - y_pred_robusto)**2).sum()
r_squared_robusto = 1 - (ssr_robusto / sst_robusto)

print("Coeficiente de Determinação (R-squared) do modelo robusto:", r_squared_robusto)

# Realizar o teste de heterocedasticidade de White
white_test_result_robusto = het_white(modelo_robusto.resid, X)
print("\nResultado do teste de heterocedasticidade de White para o modelo robusto:")
print("LM Statistic:", white_test_result_robusto[0])
print("LM p-value:", white_test_result_robusto[1])
print("F Statistic:", white_test_result_robusto[2])
print("F p-value:", white_test_result_robusto[3])


                    Robust linear Model Regression Results                    
Dep. Variable:          VM30Abril2011   No. Observations:                 1266
Model:                            RLM   Df Residuals:                     1263
Method:                          IRLS   Df Model:                            2
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Fri, 29 Sep 2023                                         
Time:                        23:49:43                                         
No. Iterations:                     2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.837e+05   8.67e+04      2.118      0.0

REGRESSÃO QUANTÍLICA_SIMPLES (NÃO LINEAR)

In [150]:
import statsmodels.api as sm

# Supondo que 'dados' seja o DataFrame com suas variáveis

# Defina as variáveis independentes e dependente
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Certifique-se de que os tipos de dados sejam numéricos
X = X.astype(float)
y = y.astype(float)

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Crie o modelo de regressão quantílica
modelo = sm.QuantReg(y, X).fit(q=0.5)  # q=0.5 representa a mediana, ajuste conforme necessário

# Visualize os resultados do modelo
print(modelo.summary())
# Faça previsões com o modelo quantílico
y_pred_quantreg = modelo.predict(X)

# Calcule o MedAE para o modelo quantílico
medae_quantreg = np.median(np.abs(y - y_pred_quantreg))

# Calcule o valor médio da variável de resposta
valor_medio_y = np.mean(y)

# Calcule o MedAE escalado pelo valor (ou preço) em percentagem
medae_scaled_quantreg = (medae_quantreg / valor_medio_y) * 100  # Em percentagem

print("Median Absolute Error (MedAE) quantílico:", medae_quantreg)
print("MedAE quantílico escalado pelo valor (ou preço) em percentagem:", medae_scaled_quantreg)


                         QuantReg Regression Results                          
Dep. Variable:          VM30Abril2011   Pseudo R-squared:               0.4629
Model:                       QuantReg   Bandwidth:                   8.926e+05
Method:                 Least Squares   Sparsity:                    3.439e+06
Date:                Fri, 29 Sep 2023   No. Observations:                 1266
Time:                        23:50:00   Df Residuals:                     1263
                                        Df Model:                            2
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.681e-06   5.23e+04  -3.21e-11      1.000   -1.03e+05    1.03e+05
PL2010         1.4483      0.005    282.753      0.000       1.438       1.458
LL2010         2.4432      0.033     73.627      0.000       2.378       2.508

The condition number is large, 1.68e+07. This might

RB_Simples (LINEAR)

In [151]:
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import KFold
from sklearn.metrics import median_absolute_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores = []

# Treinar e avaliar o modelo dentro da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo Bayesian Ridge
    modelo_bayesiano = BayesianRidge()

    modelo_bayesiano.fit(X_train, y_train)
    y_pred = modelo_bayesiano.predict(X_test)

    # Calcular o MedAE escalado pelo valor (ou preço) em percentagem
    medae = median_absolute_error(y_test, y_pred)
    valor_medio_y = np.mean(y_test)
    medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem
    medae_scores.append(medae_scaled)

    # Calcular o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

    # Visualizar os coeficientes do modelo (coeficientes Bayesianos)
    coeficientes = modelo_bayesiano.coef_
    print("Coeficientes do Modelo:", coeficientes)

# Média do MedAE escalado pelo valor (ou preço) em percentagem
mean_medae_scaled = np.mean(medae_scores)
print("Média do MedAE escalado pelo valor (ou preço) em percentagem:", mean_medae_scaled)

# Média do R²
mean_r2 = np.mean(r2_scores)
print("Média do R²:", mean_r2)


Coeficientes do Modelo: [1.61847292 1.68030638]
Coeficientes do Modelo: [1.68936412 2.05581752]
Coeficientes do Modelo: [1.79243834 1.1847947 ]
Coeficientes do Modelo: [1.64642279 1.82221011]
Coeficientes do Modelo: [1.59410037 2.15412886]
Coeficientes do Modelo: [1.76325192 1.34669725]
Coeficientes do Modelo: [1.68443074 1.67153368]
Coeficientes do Modelo: [1.58947171 2.28751832]
Coeficientes do Modelo: [1.63439239 2.21207694]
Coeficientes do Modelo: [1.6291845  2.01305308]
Média do MedAE escalado pelo valor (ou preço) em percentagem: 16.400983329402145
Média do R²: 0.6512373055411931


SVR (NÃO LINEAR)

In [152]:
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.metrics import median_absolute_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores = []

# Treinar e avaliar o modelo dentro da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo SVR com kernel RBF
    #modelo_svr = SVR(kernel='linear', C=1.0)
    modelo_svr = SVR(kernel='poly', degree=3, C=1.0)
    #modelo_svr = SVR(kernel='sigmoid', C=1.0)
    #modelo_svr = SVR(kernel='rbf', C=1.0)

    modelo_svr.fit(X_train, y_train)
    y_pred = modelo_svr.predict(X_test)

    # Calcular o MedAE escalado pelo valor (ou preço) em percentagem
    medae = median_absolute_error(y_test, y_pred)
    valor_medio_y = np.mean(y_test)
    medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem
    medae_scores.append(medae_scaled)

    # Calcular o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

# Média do MedAE escalado pelo valor (ou preço) em percentagem
mean_medae_scaled = np.mean(medae_scores)
print("Média do MedAE escalado pelo valor (ou preço) em percentagem:", mean_medae_scaled)

# Média do R²
mean_r2 = np.mean(r2_scores)
print("Média do R²:", mean_r2)


Média do MedAE escalado pelo valor (ou preço) em percentagem: 23.75966291126486
Média do R²: -0.08672074189113821


CART_simples (NÃO LINEAR)

In [153]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores = []

# Treinar e avaliar o modelo dentro da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo de árvore de decisão com os parâmetros desejados
    modelo_arvore = DecisionTreeRegressor(min_samples_split=5, random_state=42)
    #modelo_arvore = DecisionTreeRegressor(max_depth=5, min_samples_split=5, random_state=42)



    # Criar o modelo Bagging com 500 árvores
    bagging_model = BaggingRegressor(base_estimator=modelo_arvore, n_estimators=500, bootstrap=True, random_state=42)
    #bagging_model = BaggingRegressor(base_estimator=modelo_arvore, n_estimators=100, bootstrap=True, random_state=42)


    bagging_model.fit(X_train, y_train)
    y_pred = bagging_model.predict(X_test)

    # Calcular o MedAE escalado pelo valor (ou preço) em percentagem
    medae = np.median(np.abs(y_test - y_pred))
    valor_medio_y = np.mean(y_test)
    medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem
    medae_scores.append(medae_scaled)

    # Calcular o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

    # Visualizar a importância dos coeficientes (importância das variáveis)
    importancia_coeficientes = np.mean([tree.feature_importances_ for tree in bagging_model.estimators_], axis=0)
    print("Importância dos Coeficientes:", importancia_coeficientes)

# Média do MedAE escalado pelo valor (ou preço) em percentagem
mean_medae_scaled = np.mean(medae_scores)
print("Média do MedAE escalado pelo valor (ou preço) em percentagem:", mean_medae_scaled)

# Média do R²
mean_r2 = np.mean(r2_scores)
print("Média do R²:", mean_r2)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Importância dos Coeficientes: [0.87022492 0.12977508]


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Importância dos Coeficientes: [0.86110859 0.13889141]


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


...

Importância dos Coeficientes: [0.85795786 0.14204214]


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Importância dos Coeficientes: [0.86759527 0.13240473]


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Importância dos Coeficientes: [0.87041574 0.12958426]
Média do MedAE escalado pelo valor (ou preço) em percentagem: 14.229078364696914
Média do R²: 0.5720102647025542


FLORESTA ALEATÓRIA (NÃO LINEAR)

In [154]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Certifique-se de que os tipos de dados sejam numéricos
X = X.astype(float)
y = y.astype(float)

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores = []

# Treinar e avaliar o modelo dentro da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo de Floresta Aleatória com os parâmetros desejados
    random_forest_model = RandomForestRegressor(n_estimators=500, random_state=42)
    #random_forest_model = RandomForestRegressor(n_estimators=500, max_depth=10, random_state=42)
    #random_forest_model = RandomForestRegressor(n_estimators=500, max_features=4, random_state=42)
    #random_forest_model = RandomForestRegressor(n_estimators=500, max_features=0.3, random_state=42)


    random_forest_model.fit(X_train, y_train)
    y_pred = random_forest_model.predict(X_test)

    # Calcular o MedAE escalado pelo valor (ou preço) em percentagem
    medae = np.median(np.abs(y_test - y_pred))
    valor_medio_y = np.mean(y_test)
    medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem
    medae_scores.append(medae_scaled)

    # Calcular o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

    # Visualizar a importância das variáveis
    importancia_variaveis = random_forest_model.feature_importances_
    print("Importância das Variáveis:", importancia_variaveis)

# Média do MedAE escalado pelo valor (ou preço) em percentagem
mean_medae_scaled = np.mean(medae_scores)
print("Média do MedAE escalado pelo valor (ou preço) em percentagem:", mean_medae_scaled)

# Média do R²
mean_r2 = np.mean(r2_scores)
print("Média do R²:", mean_r2)


Importância das Variáveis: [0.85198725 0.14801275]
Importância das Variáveis: [0.84385787 0.15614213]
Importância das Variáveis: [0.88367251 0.11632749]
Importância das Variáveis: [0.86212843 0.13787157]
Importância das Variáveis: [0.83930719 0.16069281]
Importância das Variáveis: [0.88525781 0.11474219]
Importância das Variáveis: [0.86254208 0.13745792]
Importância das Variáveis: [0.8366682 0.1633318]
Importância das Variáveis: [0.84572728 0.15427272]
Importância das Variáveis: [0.85194658 0.14805342]
Média do MedAE escalado pelo valor (ou preço) em percentagem: 14.43210797407156
Média do R²: 0.542486201674424


GRADIENT BOOSTING (NÃO LINEAR)

In [155]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores = []

# Treinar e avaliar o modelo dentro da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo de Regressão Gradient Boosting com os parâmetros desejados
    gradient_boosting_model = GradientBoostingRegressor(n_estimators=500, random_state=42)

    gradient_boosting_model.fit(X_train, y_train)
    y_pred = gradient_boosting_model.predict(X_test)

    # Calcular o MedAE escalado pelo valor (ou preço) em percentagem
    medae = np.median(np.abs(y_test - y_pred))
    valor_medio_y = np.mean(y_test)
    medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem
    medae_scores.append(medae_scaled)

    # Calcular o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

    # Visualizar a importância das variáveis
    importancia_variaveis = gradient_boosting_model.feature_importances_
    print("Importância das Variáveis:", importancia_variaveis)

# Média do MedAE escalado pelo valor (ou preço) em percentagem
mean_medae_scaled = np.mean(medae_scores)
print("Média do MedAE escalado pelo valor (ou preço) em percentagem:", mean_medae_scaled)

# Média do R²
mean_r2 = np.mean(r2_scores)
print("Média do R²:", mean_r2)


Importância das Variáveis: [0.85935704 0.14064296]
Importância das Variáveis: [0.81805766 0.18194234]
Importância das Variáveis: [0.8754046 0.1245954]
Importância das Variáveis: [0.85055114 0.14944886]
Importância das Variáveis: [0.83842492 0.16157508]
Importância das Variáveis: [0.89175798 0.10824202]
Importância das Variáveis: [0.86364624 0.13635376]
Importância das Variáveis: [0.83163422 0.16836578]
Importância das Variáveis: [0.84431224 0.15568776]
Importância das Variáveis: [0.84499112 0.15500888]
Média do MedAE escalado pelo valor (ou preço) em percentagem: 12.699941787318881
Média do R²: 0.351145473964532


REDES NEURAIS (NÃO LINEAR)

In [156]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores = []

# Treinar e avaliar o modelo dentro da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo de regressão de rede neural com os parâmetros desejados
    modelo_rede_neural = MLPRegressor(hidden_layer_sizes=(30), max_iter=1000, random_state=42)

    modelo_rede_neural.fit(X_train, y_train)
    y_pred = modelo_rede_neural.predict(X_test)

    # Calcular o MedAE escalado pelo valor (ou preço) em percentagem
    medae = np.median(np.abs(y_test - y_pred))
    valor_medio_y = np.mean(y_test)
    medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem
    medae_scores.append(medae_scaled)

    # Calcular o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

# Média do MedAE escalado pelo valor (ou preço) em percentagem
mean_medae_scaled = np.mean(medae_scores)
print("Média do MedAE escalado pelo valor (ou preço) em percentagem:", mean_medae_scaled)

# Média do R²
mean_r2 = np.mean(r2_scores)
print("Média do R²:", mean_r2)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perce

Média do MedAE escalado pelo valor (ou preço) em percentagem: 11.691986211955475
Média do R²: 0.6298537572510516
